# Aprendizagem por reforço: jogo da velha

 Número de posições possíveis com pelo menos duas casas livres = $3^7 \times 72 = 157.464$ (número de possibilidades para um tabuleiro com $7$ casas, e $3$ possibilidades por casa - O, X e vazio - vezes o número de maneiras possíveis de escolher as duas casas livres)
 
## Modelo Beta-Binomial

Para uma certa posição $x$, com $k(x)$ casas livres, tenho $n(x)$ ações possíveis. A partir de agora, vou ignorar o $x$, mantendo sempre em mente que teremos um modelo desses abaixo para cada uma das $157.464$ posições.

Para cada uma das $k$ ações possíveis, defino uma probabilidade $\pi(i)$: a probabilidade de vitória caso o jogador escolha a a ação $i$ ($i=1,...k$)

Observo $n_i$ jogos começando da mesma posição, e escolho sempre a ação $i$. Conto o número de vitórias $v_i$. Repito isso para todas as $k$ posições; ou seja, observo $(v_i, n_i)$ para $i=1,...,k$.  

Supondo que conheço as probabilidades $\pi(i)$, a probabilidade associada ao número de jogos e vitórias observadas (a verossimilhança) é uma binomial:

$P(v_i | n_i, \pi(i)) \propto \pi(i)^{v_i}(1-\pi(i))^{n_i-v_i}$

Considerando agora todas as posições, e supondo que os resultados dos jogos são independentes (condicionalmente a $\pi$), temos a verossimilhança conjunta de todas as posições:

$P(\mathbf{v} | \mathbf{n}, \mathbf{\pi}) \propto \prod_{i=1}^k \pi(i)^{v_i}(1-\pi(i))^{n_i-v_i}$

Como não conheço $\pi$, vou modelar minha ignorância usando uma distribuição de probabilidade para esses valores (uma priori). Por questões matemáticas, escolho uma distribuição Beta para cada possível ação:

$P(\pi(i)) \propto \pi(i)^{\alpha_i - 1}(1-\pi(i))^{\beta_i-1}$

Onde $\alpha_i$ e $\beta_i$ são hiperparâmetros que vamos precisar escolher (em lugar de aprender seus valores). 

## Aprendizagem e o teorema de Bayes

Eu começo então dizendo que a probabilidade de vitória do movimento $i$ tem distribuição a priori Beta com hiperparâmetros $\alpha_i$ e $\beta_i$; jogo uma vez e escolho o movimento $i$. Como incorporo essa nova informação?

Pelo teorema de Bayes, e usando a propriedade de conjugação entre priori Beta e verossimilhança binomial, sei que a nova distribuição de $\pi(i)$ é de novo uma Beta, mas agora com parâmetros $(\alpha_i + 1, \beta_i)$ em caso de vitória, e $(\alpha_i, \beta_i+1)$ em caso de derrota. 

Portanto a equação de aprendizagem desse modelo, para cada posição $x$ e cada possível movimento $i_x$, é:

$\begin{align}
&\alpha_i(t+1) = \alpha_i(t) + r_i(t) \\
&\beta_i(t+1) = \beta_i(t) + (1-r_i(t))
\end{align}$

onde $r_i(t) = 1$ no caso de vitória, e $0$ caso contrário.

Observação: agora temos uma interpretação para os hiperparâmetros $\alpha$ e $\beta$; eles representam quantas vitórias e quantas derrotas o robô viu até o momento em que começa a nova rodada de aprendizagem.

## Decisão

O robô chegou numa posição $x$ com diversas ações possíveis. Ele tem uma distribuição de probabilidade para a probabilidade de vitória (a probabilidade de uma probabilidade...) de cada lance. O que fazer?

1. Olhar o valor esperado de probabilidade de vitória para cada movimento; escolher o movimento com maior valor esperado.
2. Sortear uma probabilidade de vitória para cada movimento, conforme a distribuição atual; escolher o movimento com maior valor sorteado

In [1]:
%matplotlib tk
from owg_board import owg
from owg_players import jb, cientista, cientista_sovina, cientista_cauteloso, miope

Inicializa jogadores

In [2]:
p1 = cientista_cauteloso()

In [18]:
p2 = cientista_cauteloso()

Reseta o tabuleiro (desnecessário no primeiro round)

In [20]:
p1.reset()
p2.reset()

### Treino por self-learning

p1 joga contra p2 por $n$ jogos

In [21]:
from tqdm import tqdm 

# Contadores
vitorias = 0
derrotas = 0
empates = 0

# Proporções de cada resultado ao longo do tempo
pempate = []
pjog1 = []
pjog2 = []

# Número de jogos para treinar
n = 100000

for i in tqdm(range(n)):
    
    # Alterna quem começa o jogo
    if i % 2 == 0:
        # Joga p1
        movimento = p1.joga()
        # Comunica o movimento para p2
        p2.comunica(movimento)
        # Checa se o jogo acabou
        res, _ = p2.board.check_result()
        while res is None:
            # Enquanto não acabar
            movimento = p2.joga()
            if movimento is not None:
                p1.comunica(movimento)
                movimento = p1.joga()           
                if movimento is not None:
                    p2.comunica(movimento)
            res, _ = p2.board.check_result()
        
        # Checa o resultado e atualiza os contadores
        r, _ = p1.board.check_result()
        if r == 1:
            vitorias += 1
        elif r == -1:
            derrotas += 1
        else:
            empates += 1
        p1.reset()
        p2.reset()
    else:
        # Joga p2
        movimento = p2.joga()
        # Comunica o movimento para p1
        p1.comunica(movimento)
        # Checa se o jogo acabou
        res, _ = p1.board.check_result()
        while res is None:
            # Enquanto não acabar
            movimento = p1.joga()
            if movimento is not None:
                p2.comunica(movimento)
                movimento = p2.joga()          
                if movimento is not None:
                    p1.comunica(movimento)
            res, _ = p1.board.check_result()
        
        # Checa o resultado final e atualiza contadores
        r, _ = p1.board.check_result()
        if r == 1:
            vitorias += 1
        elif r == -1:
            derrotas += 1
        else:
            empates += 1
        p1.reset()
        p2.reset()
        
    #print("Jogo {}: P1 {:.2%}, P2 {:.2%}, Empate {:.2%}".format(i+1, vitorias / (i+1), derrotas / (i+1), empates / (i+1)))
    pempate.append(empates / (i+1))
    pjog1.append(vitorias / (i+1))
    pjog2.append(derrotas / (i+1))

100%|██████████| 100000/100000 [10:08<00:00, 164.35it/s]


Gráfico da evoluçao das proporções de empate e vitória de cada jogador

In [22]:
from matplotlib import pyplot as plt
plt.figure(figsize = (20, 10))
plt.plot(pjog1, '-', label = p1.nome)
plt.plot(pempate, '-', label = 'Empate')
plt.plot(pjog2, '-', label = p2.nome)
plt.ylim([0,1])
plt.hlines(y = 0.5, xmin = 0, xmax = len(pjog1) )
plt.grid(True)
plt.legend()
plt.show()

Carregando job1m, o atual campeão

In [24]:
import pickle
with open('job1m.pkl', 'wb') as arq:
    pickle.dump(p1, arq)

In [ ]:
import pickle
with open('job1m.pkl', 'rb') as arq:
    campeao = pickle.load(arq)

In [23]:
# Jogar contra quem?
p = p1
tab = owg()
tab.start(p)

Empate
Empate
X ganhou
Empate
X ganhou
